Topics: Relevance Ranking
* tf-idf
* BM25
* Sentence embeddings
* Cosine similarity
* Maximal Marginal Relevance
* Spatial Similarity Computation (Area of overlap, Hausdorff distance, Earth mover's distance)

# Chapter 1: tf-idf

In [30]:
sentence1 = "This is a sentence about a city in Germany called Münster"
sentence2 = "This sentence is also about a city in Germany. It is called Berlin."
sentence3 = "Third city, but not in Germany. It is called Paris"

sentences = [sentence1, sentence2, sentence3]
sentences_names = ["sentence1", "sentence2", "sentence3"]

In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(input="content", norm = None, smooth_idf=False)
tfidf_vector = tfidf_vectorizer.fit_transform(sentences)

tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=sentences_names, columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

,about,also,berlin,but,called,city,germany,in,is,it,münster,not,paris,sentence,third,this
sentence1,1.405465,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,0.000000,2.098612,0.000000,0.000000,1.405465,0.000000,1.405465
sentence2,1.405465,2.098612,2.098612,0.000000,1.0,1.0,1.0,1.0,2.0,1.405465,0.000000,0.000000,0.000000,1.405465,0.000000,1.405465
sentence3,0.000000,0.000000,0.000000,2.098612,1.0,1.0,1.0,1.0,1.0,1.405465,0.000000,2.098612,2.098612,0.000000,2.098612,0.000000


In [31]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
count_vector = count_vectorizer.fit_transform(sentences)

count_df = pd.DataFrame(count_vector.toarray(), index=sentences_names, columns=count_vectorizer.get_feature_names_out())
count_df

,about,also,berlin,but,called,city,germany,in,is,it,münster,not,paris,sentence,third,this
sentence1,1,0,0,0,1,1,1,1,1,0,1,0,0,1,0,1
sentence2,1,1,1,0,1,1,1,1,2,1,0,0,0,1,0,1
sentence3,0,0,0,1,1,1,1,1,1,1,0,1,1,0,1,0


In [58]:
import math

# is, for document 1
n = 3
df = 3
tf = 2


# it, for document 1
n = 3
df = 2
tf = 1

# berlin, for sentence 2
n = 3
df = 1
tf = 1

idf = math.log(n/df) + 1 # math.log returns the natural logarithm

tf_idf = tf * idf
tf_idf

2.09861228866811

Computing similarity scores for an input sentence

In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
docs_tfidf = vectorizer.fit_transform(sentences)

def get_tf_idf_query_similarity(vectorizer, docs_tfidf, query):
    """
    vectorizer: TfIdfVectorizer model
    docs_tfidf: tfidf vectors for all docs
    query: query doc

    return: cosine similarity between query and all docs
    """
    query_tfidf = vectorizer.transform([query])
    cosineSimilarities = cosine_similarity(query_tfidf, docs_tfidf).flatten()
    return query_tfidf, cosineSimilarities

In [60]:
query = "Tell me something about Münster, Germany. I know nothing about it"

query_tfidf, similarities = get_tf_idf_query_similarity(vectorizer, docs_tfidf, query)
similarities

array([0.57488947, 0.37968183, 0.17906684])

In [61]:
modified_query = "about it me about something Münster, Germany. Tell I know nothing"

query_tfidf, similarities = get_tf_idf_query_similarity(vectorizer, docs_tfidf, modified_query)
similarities

array([0.57488947, 0.37968183, 0.17906684])

# Chapter 2: BM25

In [36]:
!pip install --quiet rank_bm25 # https://pypi.org/project/rank-bm25/

In [37]:
sentence1 = "This is a sentence about a city in Germany called Münster"
sentence2 = "This sentence is also about a city in Germany. It is called Berlin."
sentence3 = "Third city, but not in Germany. It is called Paris"

sentences = [sentence1, sentence2, sentence3]
sentences_names = ["sentence1", "sentence2", "sentence3"]

In [62]:
from rank_bm25 import BM25Okapi

tokenized_corpus = [doc.split(" ") for doc in sentences]

bm25 = BM25Okapi(tokenized_corpus)
print(tokenized_corpus)

[['This', 'is', 'a', 'sentence', 'about', 'a', 'city', 'in', 'Germany', 'called', 'Münster'], ['This', 'sentence', 'is', 'also', 'about', 'a', 'city', 'in', 'Germany.', 'It', 'is', 'called', 'Berlin.'], ['Third', 'city,', 'but', 'not', 'in', 'Germany.', 'It', 'is', 'called', 'Paris']]


In [63]:
query = "Tell me something about Münster, Germany. I know nothing about it"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

array([-0.12843885, -0.17830856, -0.06691186])

In [64]:
modified_query = "about it me about something Münster, Germany. Tell I know nothing"

tokenized_query_modified = modified_query.split(" ")

doc_scores = bm25.get_scores(tokenized_query_modified)
doc_scores

array([-0.12843885, -0.17830856, -0.06691186])

# Chapter 3: Sentence Embeddings

Embeddings with Sentence Transformer

In [41]:
pip install -qU sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 4.5 MB/s eta 0:00:00


In [65]:
# The sentences to encode
sentence1 = "This is a sentence about a city in Germany called Münster"
sentence2 = "This sentence is also about a city in Germany. It is called Berlin."
sentence3 = "Third city, but not in Germany. It is called Paris"
sentences = [sentence1, sentence2, sentence3]

# The queries to encode
query = "Tell me something about Münster, Germany. I know nothing about it"
modified_query = "about it me about something Münster, Germany. Tell I know nothing"
surprise_query = "This is related to a place outside the country"

In [66]:
# https://sbert.net/docs/sentence_transformer/pretrained_models.html
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-mpnet-base-v2") # all-MiniLM-L6-v2

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 768]

query_embedding = model.encode(query)
modified_query_embedding = model.encode(modified_query)
surprise_query_embedding = model.encode(surprise_query)

# 3. Calculate the embedding similarities
similarities1 = model.similarity(embeddings, query_embedding)
print(similarities1)

similarities2 = model.similarity(embeddings, modified_query_embedding)
print(similarities2)

similarities3 = model.similarity(embeddings, surprise_query_embedding)
print(similarities3)

similarities4 = model.similarity(query_embedding, modified_query_embedding)
print(similarities4)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(3, 768)
tensor([[0.7714],
        [0.5006],
        [0.4581]])
tensor([[0.6956],
        [0.5296],
        [0.4308]])
tensor([[0.4664],
        [0.5410],
        [0.3139]])
tensor([[0.8056]])


Embeddings with OpenAI

In [67]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [43]:
!pip install -q openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [68]:
# The sentences to encode
sentence1 = "This is a sentence about a city in Germany called Münster"
sentence2 = "This sentence is also about a city in Germany. It is called Berlin."
sentence3 = "Third city, but not in Germany. It is called Paris"
sentences = [sentence1, sentence2, sentence3]

# The queries to encode
query = "Tell me something about Münster, Germany. I know nothing about it"
modified_query = "about it me about something Münster, Germany. Tell I know nothing"
surprise_query = "This is related to a place outside the country"

In [70]:
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

client = OpenAI()

# 1. Calculate embeddings using the OpenAI model

def create_openai_embeddings (input_text):
    response = client.embeddings.create(
        input= input_text,
        model="text-embedding-3-large" # text-embedding-3-small or text-embedding-3-large
    )
    return response

response =  create_openai_embeddings(sentences)
embeddings =  [result.embedding for result in response.data]

query_embedding = create_openai_embeddings(query).data[0].embedding
modified_query_embedding = create_openai_embeddings(modified_query).data[0].embedding
surprise_query_embedding = create_openai_embeddings(surprise_query).data[0].embedding

# print the shape of an embedding as an example
print(np.asarray(query_embedding).shape)

# 2. Calculate the embedding similarities
similarities1 = cosine_similarity(embeddings, [query_embedding])
print(similarities1)

similarities2 = cosine_similarity(embeddings, [modified_query_embedding])
print(similarities2)

similarities3 = cosine_similarity(embeddings, [surprise_query_embedding])
print(similarities3)

similarities4 = cosine_similarity([query_embedding], [modified_query_embedding])
print(similarities4)

(3072,)
[[0.70099343]
 [0.37329495]
 [0.32497704]]
[[0.57735339]
 [0.40868865]
 [0.33003281]]
[[0.21165781]
 [0.2490101 ]
 [0.25942753]]
[[0.81293973]]


# Chapter 4: Cosine Similarity

In [55]:
import math

X = [[0, 5, 0]]
Y = [[2, 7, 0]] #, [0, 5, 0], [2, 2, 2]] # the length of the vector does not matter
Z = [[5, 0, 0]]

from sklearn.metrics.pairwise import cosine_similarity

print("XY similarity:", cosine_similarity(X, Y).flatten(), "angle:", math.degrees(math.acos(cosine_similarity(X, Y).flatten()[0])))
print("YZ similarity:", cosine_similarity(Y, Z).flatten(), "angle:", math.degrees(math.acos(cosine_similarity(Y, Z).flatten()[0])))
print("XZ similarity:", cosine_similarity(X, Z).flatten(), "angle:", math.degrees(math.acos(cosine_similarity(X, Z).flatten()[0])))

XY similarity: [0.96152395] angle: 15.945395900922858
YZ similarity: [0.27472113] angle: 74.05460409907715
XZ similarity: [0.] angle: 90.0


# Chapter 5: Maximal Marginal Relevance

In [1]:
!pip install --quiet langchain langchain_openai langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.3 MB/s eta 0:00:00


In [4]:
# The sentences to encode
sentence1 = "This is a sentence about a city in Germany called Münster"
sentence2 = "This sentence is also about a city in Germany. It is called Berlin."
sentence3 = "Third city, but not in Germany. It is called Paris"
sentences = [sentence1, sentence2, sentence3]

# The queries to encode
query = "Tell me something about Münster, Germany. I know nothing about it"
modified_query = "about it me about something Münster, Germany. Tell I know nothing"
surprise_query = "This is related to a place outside the country"

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the sentences into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5, chunk_overlap=0)

chunks = text_splitter.create_documents(sentences)

print (f"The sentences have been split into {len(chunks)} chunks")
chunks

The sentences have been split into 47 chunks


[Document(metadata={}, page_content='This'),
 Document(metadata={}, page_content='is a'),
 Document(metadata={}, page_content='sent'),
 Document(metadata={}, page_content='ence'),
 Document(metadata={}, page_content='abou'),
 Document(metadata={}, page_content='t'),
 Document(metadata={}, page_content='a'),
 Document(metadata={}, page_content='city'),
 Document(metadata={}, page_content='in'),
 Document(metadata={}, page_content='Germ'),
 Document(metadata={}, page_content='any'),
 Document(metadata={}, page_content='call'),
 Document(metadata={}, page_content='ed'),
 Document(metadata={}, page_content='Müns'),
 Document(metadata={}, page_content='ter'),
 Document(metadata={}, page_content='This'),
 Document(metadata={}, page_content='sent'),
 Document(metadata={}, page_content='ence'),
 Document(metadata={}, page_content='is'),
 Document(metadata={}, page_content='also'),
 Document(metadata={}, page_content='abou'),
 Document(metadata={}, page_content='t'),
 Document(metadata={}, page

In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=chunks, embedding=embedding, collection_name = "col")

retriever_simple = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 8})
retriever_mmr = vectordb.as_retriever(search_type="mmr", search_kwargs={"k": 8})

In [7]:
relevant_docs = retriever_simple.invoke("ent")
relevant_docs

[Document(metadata={}, page_content='ence'),
 Document(metadata={}, page_content='ence'),
 Document(metadata={}, page_content='ter'),
 Document(metadata={}, page_content='sent'),
 Document(metadata={}, page_content='sent'),
 Document(metadata={}, page_content='in'),
 Document(metadata={}, page_content='in'),
 Document(metadata={}, page_content='in')]

In [8]:
mmr_relevant_docs = retriever_mmr.invoke("ent")
mmr_relevant_docs

[Document(metadata={}, page_content='ence'),
 Document(metadata={}, page_content='ter'),
 Document(metadata={}, page_content='sent'),
 Document(metadata={}, page_content='in'),
 Document(metadata={}, page_content='ed'),
 Document(metadata={}, page_content='t'),
 Document(metadata={}, page_content='not'),
 Document(metadata={}, page_content='abou')]

# Chapter 6: Spatial Similarity Computation

## Retrieve and visualize the data

In [3]:
!pip install --quiet turfpy

In [72]:
import requests
from turfpy.transformation import circle

url = "https://raw.githubusercontent.com/aurioldegbelo/sis2024/refs/heads/main/vector_data/geo1_example.json"

data = requests.get(url).json()

geo1 = data["features"][0]
denkpause = data["features"][1]
king_kebab = data["features"][2]
krimphove = data["features"][3]
ifgi = data["features"][4]

print(geo1)
print(denkpause)
print(king_kebab)
print(krimphove)
print(ifgi)

q_footprint = circle(ifgi, radius=100,  steps=10,  units='m')
q_footprint['properties'] = ifgi['properties']

target_footprints = [geo1, denkpause, king_kebab, krimphove]

print(q_footprint)
print(target_footprints)

{'type': 'Feature', 'properties': {'Name': 'GEO1', 'id': 0}, 'geometry': {'coordinates': [[[7.595410964969801, 51.96985640112052], [7.595072152923166, 51.96894953936396], [7.596296431749778, 51.96883201435048], [7.596259418668183, 51.96931965343859], [7.596276501629234, 51.96956522507139], [7.596225252747445, 51.969682748162], [7.596020257222932, 51.9697827302507], [7.595410964969801, 51.96985640112052]]], 'type': 'Polygon'}}
{'type': 'Feature', 'properties': {'Name': 'Bistro Denkpause', 'id': 1}, 'geometry': {'coordinates': [[[7.59442844476078, 51.96876957052007], [7.594415375105655, 51.968695759584904], [7.5945373585494735, 51.96869039151193], [7.594480723379263, 51.96832670308601], [7.594731225092858, 51.968313282792536], [7.594837960605048, 51.96874675624372], [7.59442844476078, 51.96876957052007]]], 'type': 'Polygon'}}
{'type': 'Feature', 'properties': {'Name': 'King Kebab', 'id': 2}, 'geometry': {'coordinates': [[[7.596912214932445, 51.969501466701416], [7.596895026350126, 51.969

In [73]:
# https://sustainability-gis.readthedocs.io/en/latest/lessons/L1/intro-to-python-geostack.html
import shapely # geometric operations
import folium # rendering and visualization on the map

map = folium.Map(location=[51.969,  7.595], zoom_start="17")

popup = folium.GeoJsonPopup(fields=["Name"])

folium.GeoJson(data, name="examples", popup = popup).add_to(map)
folium.LayerControl().add_to(map)

map

## Area of overlap

In [42]:
!pip install --quiet turfpy

In [144]:
# What are the boundaries of the ifgi building?
from turfpy.measurement import area
from turfpy.transformation import intersect, circle, union
from geojson import FeatureCollection
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler


def area_of_overlaps(q_footprint, target_footprints):
    lst = []
    for target in target_footprints:
      aoo = 0
      if intersect([q_footprint, target]) != None:
         aoo = area(intersect([q_footprint, target]))
         aou = area(union(FeatureCollection([q_footprint, target])))
      new_row = {'qf': q_footprint['properties'],
                 'tf': target['properties'],
                 'area_of_overlap': aoo,
                 'intersection_over_union': aoo/aou}
      lst.append(new_row)
    return pd.DataFrame(lst)

df = area_of_overlaps(q_footprint, target_footprints)

# Brief intro to normalization in general at https://www.codecademy.com/article/normalization
# Normalization using MinMaxScaler https://scikit-learn.org/1.5/modules/generated/sklearn.preprocessing.MinMaxScaler.html
x_array = np.array(df['area_of_overlap'])
print("Array for area of overlaps: ", x_array)
scaler = MinMaxScaler()
expanded_array = np.expand_dims(x_array, axis=1) # we need to add a dimension to be able to use MinMaxScaler
normalized_arr = scaler.fit_transform(expanded_array)
#print(x_array.shape)
#print(expanded_array.shape)
#print(np.squeeze(normalized_arr).shape)
#print("Normalized Array for area of overlaps: ", normalized_arr)

df['normalized_score'] = np.squeeze(normalized_arr) # we need to remove a dimension after the scaling operation
df

Array for area of overlaps:  [7227.35873859    0.          100.28670279    0.        ]


/usr/local/lib/python3.10/dist-packages/turfpy/transformation.py:283: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  result = cascaded_union(shapes)


,qf,tf,area_of_overlap,intersection_over_union,normalized_score
0,"{'Name': 'ifgi', 'id': 4}","{'Name': 'GEO1', 'id': 0}",7227.358739,0.245331,1.000000
1,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Bistro Denkpause', 'id': 1}",0.000000,0.000000,0.000000
2,"{'Name': 'ifgi', 'id': 4}","{'Name': 'King Kebab', 'id': 2}",100.286703,0.003404,0.013876
3,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Krimphove', 'id': 3}",0.000000,0.000000,0.000000


In [141]:
from geojson import Point, Feature
from ipyleaflet import Map, GeoJSON
from turfpy.transformation import circle

center = [51.96954,  7.595]

m = Map(center=center, zoom_start=19)

qf_layer = GeoJSON(data=q_footprint, style={"color": "green"})
target_layer = GeoJSON(data=data)

m.add_layer(qf_layer)

m.add_layer(target_layer)

m

Map(center=[51.96954, 7.595], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

## Hausdorff distance

In [23]:
from turfpy.measurement import area, bbox, bbox_polygon
from geojson import Polygon
from turfpy.transformation import intersect
from shapely import hausdorff_distance
from shapely import polygons

# hausdorf distance computation needs inputs of the same size, hence direct computation over the raw polygons is not possible
def compute_hd_distance_on_bbox (q_footprint, target_footprint):

  bbox_q = bbox_polygon(bbox(q_footprint)) # create a bounding box (coordinates) and then the bounding box (polygon)
  bbox_q.properties=q_footprint['properties'] # update the property values of the new polygon

  bbox_t = bbox_polygon(bbox(target_footprint)) # create a bounding box (coordinates) and then the bounding box (polygon)
  bbox_t.properties=target_footprint['properties'] # update the property values of the new polygon

  # get the coordinates of the bounding box (all four points)
  u = (bbox_q['geometry'].coordinates)[0]
  v = (bbox_t['geometry'].coordinates)[0]

  # create polygons
  p1 = polygons(u)
  p2 = polygons(v)

  # compute and return the area of overlap as well, for comparison
  aoo = 0
  if intersect([bbox_q, bbox_t]) != None:
     aoo = area(intersect([bbox_q, bbox_t]))

 # return directed_hausdorff(u, v)[0], aoo
  return hausdorff_distance(p1, p2), aoo

def compute_hd_distance (q_footprint, target_footprint):

  u = (q_footprint['geometry'].coordinates)[0]
  v = (target_footprint['geometry']['coordinates'])[0]

  p1 = polygons(u)
  p2 = polygons(v)

  # compute and return the area of overlap as well, for comparison
  aoo = 0
  if intersect([q_footprint, target_footprint]) != None:
      aoo = area(intersect([q_footprint, target_footprint]))

  return hausdorff_distance(p1, p2), aoo

In [25]:
from turfpy.measurement import area
from turfpy.transformation import intersect, circle
from scipy.spatial.distance import directed_hausdorff
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd

def hausdorff_distances_on_bbox(q_footprint, target_footprints):
    lst = []
    for target in target_footprints:
      hdd = compute_hd_distance_on_bbox(q_footprint, target)
      new_row = {'qf': q_footprint['properties'],
                 'tf': target['properties'],
                 'hausdorff_distance': hdd[0],
                 'area_of_overlap': hdd[1]}
      lst.append(new_row)
    return pd.DataFrame(lst)

def hausdorff_distances(q_footprint, target_footprints):
    lst = []
    for target in target_footprints:
      hdd = compute_hd_distance(q_footprint, target)
      new_row = {'qf': q_footprint['properties'],
                 'tf': target['properties'],
                 'hausdorff_distance': hdd[0],
                 'area_of_overlap': hdd[1]}
      lst.append(new_row)
    return pd.DataFrame(lst)


df2 = hausdorff_distances_on_bbox(q_footprint, target_footprints)

x_array2 = np.array(df2['hausdorff_distance'])
print("Array for hausdorff distances: ", x_array2)
scaler = MinMaxScaler(feature_range=(-1, 0))
expanded_array2 = np.expand_dims(x_array2, axis=1) # we need to add a dimension to be able to use MinMaxScaler
normalized_arr2 = scaler.fit_transform(expanded_array2)

df2['normalized_score'] = normalized_arr2 * (-1)
df2

Array for hausdorff distances:  [0.00112198 0.00293948 0.00262151 0.00305963]


,qf,tf,hausdorff_distance,area_of_overlap,normalized_score
0,"{'Name': 'ifgi', 'id': 4}","{'Name': 'GEO1', 'id': 0}",0.001122,9568.922329,1.000000
1,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Bistro Denkpause', 'id': 1}",0.002939,234.627234,0.062011
2,"{'Name': 'ifgi', 'id': 4}","{'Name': 'King Kebab', 'id': 2}",0.002622,178.830352,0.226112
3,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Krimphove', 'id': 3}",0.003060,0.000000,-0.000000


In [74]:
from geojson import Point, Feature
from ipyleaflet import Map, GeoJSON, LayersControl
from turfpy.transformation import circle

center = [51.96954, 7.595]

m = Map(center=center, zoom_start=19)

qbbox_layer = GeoJSON(name="query bbox", data=bbox_polygon(bbox(q_footprint)), style={"color": "green"})
target_bbox_layer = GeoJSON(name = "raw data layer", data=data)

geo1_bbox_layer = GeoJSON(name = "geo1 bbox", data=bbox_polygon(bbox(geo1)) , style={"color": "red"})
denkpause_bbox_layer = GeoJSON(name = "denkpause bbox", data=bbox_polygon(bbox(denkpause)) , style={"color": "red"})
king_kebab_bbox_layer = GeoJSON(name = "king kebab bbox ", data=bbox_polygon(bbox(king_kebab)) , style={"color": "red"})
krimphove_bbox_layer = GeoJSON(name ="krimphove bbox", data=bbox_polygon(bbox(krimphove)) , style={"color": "red"})


control = LayersControl(position="topright")
m.add_control(control)

m.add_layer(target_bbox_layer)
m.add_layer(qbbox_layer)
m.add_layer(geo1_bbox_layer)
m.add_layer(denkpause_bbox_layer)
m.add_layer(king_kebab_bbox_layer)
m.add_layer(krimphove_bbox_layer)

m

Map(center=[51.96954, 7.595], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

# Earth Mover's distance

In [57]:
from scipy.stats import wasserstein_distance_nd

#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wasserstein_distance_nd.html#scipy.stats.wasserstein_distance_nd
def compute_wasserstein_distance (q_footprint, target_footprint):

  u = (q_footprint['geometry'].coordinates)[0]
  v = (target_footprint['geometry']['coordinates'])[0]

  return wasserstein_distance_nd(u, v)

def earth_mover_distances(q_footprint, target_footprints):
    lst = []
    for target in target_footprints:
      wd = compute_wasserstein_distance(q_footprint, target)

      aoo = 0
      if intersect([q_footprint, target]) != None:
         aoo = area(intersect([q_footprint, target]))
      new_row = {'qf': q_footprint['properties'],
                 'tf': target['properties'],
                 'wasserstein_distance': wd,
                 'area_of_overlap':aoo}
      lst.append(new_row)
    return pd.DataFrame(lst)


df3 = earth_mover_distances(q_footprint, target_footprints)
df3

# normalization
x3_array = np.array(df3['wasserstein_distance'])
print("Array for earth mover's distances: ", x3_array)
scaler = MinMaxScaler(feature_range=(-1, 0))
expanded_array3 = np.expand_dims(x3_array, axis=1) # we need to add a dimension to be able to use MinMaxScaler
normalized_arr3 = scaler.fit_transform(expanded_array3)

df3['normalized_score'] = normalized_arr3 * (-1)
df3

Array for earth mover's distances:  [0.00073008 0.00179195 0.00140664 0.00178281]


,qf,tf,wasserstein_distance,area_of_overlap,normalized_score
0,"{'Name': 'ifgi', 'id': 4}","{'Name': 'GEO1', 'id': 0}",0.000730,7227.358739,1.000000
1,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Bistro Denkpause', 'id': 1}",0.001792,0.000000,-0.000000
2,"{'Name': 'ifgi', 'id': 4}","{'Name': 'King Kebab', 'id': 2}",0.001407,100.286703,0.362861
3,"{'Name': 'ifgi', 'id': 4}","{'Name': 'Krimphove', 'id': 3}",0.001783,0.000000,0.008611


# Project work

* Exercice 01: pick a method for topical similarity computation
* Exercice 02: pick a method for spatial similarity computation